In [1]:
%reload_ext autoreload
%autoreload 2

import os
import pandas as pd

query_path = "/home/wuyinjun/concept_based_retrieval/"

img_caption_file_name= os.path.join(query_path, "prod_hard_negatives/prod_vg_hard_negs_swap_all.csv")

caption_pd = pd.read_csv(img_caption_file_name)

In [2]:
import torch
from transformers import CLIPModel, AutoProcessor


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# processor = ViTImageProcessor.from_pretrained('google/vit-large-patch16-224')
# model = ViTForImageClassification.from_pretrained('google/vit-large-patch16-224').to(device)
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").to(device)
# processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
raw_processor = AutoProcessor.from_pretrained("openai/clip-vit-large-patch14")
# processor =  lambda images: raw_processor(images=images, return_tensors="pt", padding=False, do_resize=False, do_center_crop=False)["pixel_values"]
processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
text_processor =  lambda text: raw_processor(text=[text], return_tensors="pt", padding=True, truncation=True)
img_processor =  lambda images: raw_processor(images=images, return_tensors="pt")["pixel_values"]
model = model.eval()

/home/wuyinjun/miniconda3/envs/myenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/wuyinjun/miniconda3/envs/myenv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
caption_ls = list(caption_pd["caption"])
id_ls = list(caption_pd["image_id"])


text_feat_ls = []
visited_id_ls = []
visited_caption_ls = []
with torch.no_grad():

    for idx in range(len(caption_ls)):
        if id_ls[idx] in visited_id_ls:
            continue
        caption = caption_ls[idx]
        inputs = text_processor(caption)
        inputs = {key: val.to(device) for key, val in inputs.items()}
        # print(idx)
        text_features = model.get_text_features(**inputs)
        text_feat_ls.append(text_features.cpu().view(-1))
        visited_id_ls.append(id_ls[idx])
        visited_caption_ls.append(caption)
    
text_feat_tensor = torch.stack(text_feat_ls, dim=0)




In [9]:
from sklearn.cluster import KMeans
from sklearn.cluster import Birch

print(len(text_feat_tensor))
clustering = KMeans(n_clusters=500).fit(text_feat_tensor.detach().numpy())
# clustering = Birch(threshold=0.9, n_clusters=None).fit(text_feat_tensor.cpu().numpy())

4435


/home/wuyinjun/miniconda3/envs/myenv/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [10]:
clustering.labels_

array([138, 158,  37, ..., 236, 247, 193], dtype=int32)

In [11]:
# print(clustering.labels_)
import numpy as np
unique_label_ids=np.unique(np.array(clustering.labels_))
label_count_ls = [0 for _ in range(len(unique_label_ids))]

for label in clustering.labels_:
    label_count_ls[label] += 1

label_count_tensor = torch.tensor(label_count_ls)

sorted_label_ids = torch.argsort(label_count_tensor, descending=False)


# print(torch.nonzero(label_count_tensor.view(-1) > 1).view(-1))

print(label_count_tensor)

print()



tensor([ 4,  6,  8,  4, 37,  8, 13, 11,  6, 15, 22,  8,  8, 14, 19, 15, 16,  6,
        17,  2,  2, 12, 14, 15,  4, 17,  3, 10,  6,  8,  8,  7, 23,  4,  8, 17,
        23,  4,  7, 17,  9, 16, 14,  7, 17,  4, 10, 14, 11,  7,  9,  4, 13, 10,
         1, 10, 19,  3, 10,  7,  3,  2,  5, 15, 13,  8, 14,  9,  7,  3, 10, 10,
         5, 35, 14, 18,  3,  4,  5,  5, 12,  5,  3,  9, 15,  7, 10,  6, 10, 30,
        13, 15, 20, 22, 12,  9, 18, 25,  6,  4, 22, 14, 13,  9, 14, 16,  8,  6,
         5,  9, 14,  6,  8, 14,  4,  6, 11,  7,  9, 12,  7,  5, 12,  5, 11,  9,
        11, 10,  8,  8,  8,  7,  8,  6,  4, 13,  9, 30, 16,  9,  9,  1,  9, 28,
         8,  7,  9, 12, 17, 25, 16,  9, 16, 16, 18, 14, 11, 13, 30,  1, 12,  5,
        10,  7,  4,  4,  2,  9,  7, 14, 37,  7,  1,  4, 22,  5,  8,  8,  5, 20,
         4,  4, 14, 14,  4, 16,  4,  8,  6, 18,  8, 10,  7,  8,  5,  6,  4, 12,
        14,  8, 13, 10, 16,  6,  9,  4, 22, 13,  6,  5,  7, 10,  4,  7,  7,  8,
         3, 14,  5,  7,  1,  6, 20,  9, 

In [13]:
label_id_ls = torch.nonzero(label_count_tensor.view(-1) == 1).view(-1).tolist()
label_array = torch.tensor(clustering.labels_)
for label_id in label_id_ls:
    sample_ids = torch.nonzero(label_array == label_id).view(-1).tolist()
    for sample_id in sample_ids:
        print(visited_id_ls[sample_id], visited_caption_ls[sample_id])


2359340 a dog behind a boot and a shoe. the shoe has a shoelace. the dog has a head and nose.
2366126 a clock with a design under the hand and a number on it, on the side of a building with a statue inside
2360386 a dude with a tattoo on his arm holding a cell phone in his arm next to a box with a blue lighter.
2369881 photo of pants, another pants, a frisbee, and a brown shoe. there is a girl wearing a green coat.
2361534 there is a crack in the wall and a crack in the sidewalk. a woman is standing next to another woman. the first woman has hair.
2367935 bare trees on trail. trail has tracks. sign on tree. snow has tracks. snow pants are for snow.
2359509 a green light on a speaker that has a light. there is a cat laying on a desk. the cat has eyes.
2371219 pen, blue flowers, and almonds on a plate with a pie
2360526 photo of statues, another statues, and another statues, with an angel next to a circle
2365780 person wearing blue jeans with a folder holding papers and legs on a bed
23

In [14]:
visited_labels = set()

selected_img_id_ls = []

for idx in range(len(clustering.labels_)):
    label = clustering.labels_[idx]
    if label in visited_labels:
        continue
    
    selected_img_id_ls.append(visited_id_ls[idx])
    
    visited_labels.add(label)

print(len(selected_img_id_ls))
print(selected_img_id_ls)

500
[1, 5, 6, 7, 8, 9, 11, 13, 16, 23, 27, 30, 31, 34, 37, 38, 39, 40, 43, 47, 48, 53, 58, 60, 62, 64, 69, 72, 74, 79, 82, 97, 98, 130, 156, 166, 171, 179, 187, 194, 206, 221, 236, 239, 243, 246, 252, 265, 272, 277, 286, 291, 303, 313, 314, 320, 356, 368, 369, 375, 397, 414, 442, 444, 453, 462, 479, 492, 494, 502, 508, 524, 534, 538, 545, 556, 559, 560, 568, 578, 586, 597, 608, 621, 626, 630, 675, 681, 696, 697, 701, 714, 736, 738, 748, 759, 833, 870, 879, 889, 913, 917, 930, 934, 939, 940, 951, 955, 956, 964, 968, 986, 989, 991, 994, 997, 1001, 1015, 1017, 1020, 1027, 1034, 1040, 1042, 1043, 1053, 1057, 1066, 1090, 1091, 1114, 1115, 1129, 1145, 1150, 1159, 1170, 1179, 1183, 1184, 1185, 1196, 1199, 1216, 1233, 1241, 1269, 1270, 1285, 1314, 1321, 1326, 1327, 1339, 1345, 1346, 1351, 1373, 1378, 1392, 1466, 1473, 1501, 1528, 1532, 1549, 1891, 1925, 1928, 2066, 2114, 2275, 2480, 2643, 2704, 3114, 3173, 3367, 3371, 3433, 3548, 3587, 3616, 3636, 3764, 3766, 3784, 3851, 3914, 3996, 4126, 4167

In [19]:
import pickle

with open("prod_hard_negatives/selected_img_id_ls", "wb") as f:
    pickle.dump(selected_img_id_ls, f)